# Folio Quickstart: Single-Objective Optimization

This notebook demonstrates a complete Bayesian optimization workflow using Folio.

We'll optimize a synthetic 2D function with a known optimum, showing how Folio's
Bayesian optimization efficiently finds the best conditions.

## Setup

Import Folio and create a temporary database for this demo.

In [ ]:
import tempfile
import numpy as np
import matplotlib.pyplot as plt

from folio.api import Folio
from folio.core.config import TargetConfig
from folio.core.schema import InputSpec, OutputSpec

In [ ]:
# Create a temporary database
db_path = tempfile.mktemp(suffix=".db")
folio = Folio(db_path=db_path)
print(f"Database: {db_path}")

## Define the Synthetic Function

We'll use a 2D quadratic function with:
- Inputs: `x1` in [0, 10] and `x2` in [0, 10]
- Output: `yield` = 100 - (x1 - 7)^2 - (x2 - 3)^2
- **True optimum**: x1=7, x2=3 with yield=100

This simulates a reaction yield that depends on two process parameters.

In [ ]:
def synthetic_experiment(x1: float, x2: float) -> float:
    """Simulated experiment: 2D quadratic with optimum at (7, 3)."""
    noise = np.random.normal(0, 0.5)
    return 100 - (x1 - 7)**2 - (x2 - 3)**2 + noise

# True optimum
TRUE_OPTIMUM = {"x1": 7.0, "x2": 3.0}
TRUE_YIELD = 100.0

## Create a Project

Define the experimental schema with inputs, outputs, and optimization target.

In [ ]:
folio.create_project(
    name="yield_optimization",
    inputs=[
        InputSpec("x1", "continuous", bounds=(0.0, 10.0)),
        InputSpec("x2", "continuous", bounds=(0.0, 10.0)),
    ],
    outputs=[OutputSpec("yield")],
    target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
)
print("Project created!")

## Run the Optimization Loop

The workflow:
1. Get a suggestion from Folio
2. Run the experiment (here: call our synthetic function)
3. Record the observation
4. Repeat

The first few suggestions will be random (exploration phase). Once enough data
is collected, Folio uses Bayesian optimization to suggest promising conditions.

In [ ]:
N_ITERATIONS = 15
history = []  # Track progress

for i in range(N_ITERATIONS):
    # Get suggestion
    suggestion = folio.suggest("yield_optimization")[0]
    x1, x2 = suggestion["x1"], suggestion["x2"]
    
    # Run experiment
    result = synthetic_experiment(x1, x2)
    
    # Record observation
    folio.add_observation(
        project_name="yield_optimization",
        inputs={"x1": x1, "x2": x2},
        outputs={"yield": result},
    )
    
    # Track history
    history.append({"iteration": i + 1, "x1": x1, "x2": x2, "yield": result})
    print(f"Iter {i+1:2d}: x1={x1:.2f}, x2={x2:.2f} -> yield={result:.2f}")

## Analyze Results

Let's see how quickly the optimizer found good conditions.

In [ ]:
# Get all observations
observations = folio.get_observations("yield_optimization")
yields = [obs.outputs["yield"] for obs in observations]

# Best observation
best_idx = np.argmax(yields)
best_obs = observations[best_idx]
print(f"Best result: yield={yields[best_idx]:.2f}")
print(f"  at x1={best_obs.inputs['x1']:.2f}, x2={best_obs.inputs['x2']:.2f}")
print(f"\nTrue optimum: yield={TRUE_YIELD} at x1={TRUE_OPTIMUM['x1']}, x2={TRUE_OPTIMUM['x2']}")

## Plot Optimization Progress

Track how the best-found value improves over iterations.

In [ ]:
# Compute running best
running_best = np.maximum.accumulate(yields)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Left: optimization progress
ax1.plot(range(1, len(yields) + 1), running_best, "b-o", label="Best found")
ax1.axhline(TRUE_YIELD, color="r", linestyle="--", label="True optimum")
ax1.set_xlabel("Iteration")
ax1.set_ylabel("Yield")
ax1.set_title("Optimization Progress")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: sampled points in input space
x1_vals = [obs.inputs["x1"] for obs in observations]
x2_vals = [obs.inputs["x2"] for obs in observations]
colors = range(len(observations))

sc = ax2.scatter(x1_vals, x2_vals, c=colors, cmap="viridis", s=60)
ax2.scatter([TRUE_OPTIMUM["x1"]], [TRUE_OPTIMUM["x2"]], c="red", marker="*", s=200, label="True optimum")
ax2.set_xlabel("x1")
ax2.set_ylabel("x2")
ax2.set_title("Sampled Points (color = iteration)")
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.legend()
plt.colorbar(sc, ax=ax2, label="Iteration")

plt.tight_layout()
plt.show()

## Summary

This demo showed the core Folio workflow:

1. **Create a project** with input/output schema and optimization target
2. **Get suggestions** using `folio.suggest()`
3. **Record results** using `folio.add_observation()`
4. **Repeat** - Folio's Bayesian optimization learns from data

The optimizer efficiently explores the input space and converges toward the optimum,
typically finding good conditions with far fewer experiments than random search.

In [ ]:
# Cleanup: delete the project
folio.delete_project("yield_optimization")
print("Demo complete!")